# Projet HAI923 - Modèle CLIP Image-Texte

**Nom:** [À COMPLÉTER]  
**Prénom:** [À COMPLÉTER]  
**N° Carte Étudiant:** [À COMPLÉTER]  
**Numéro de Groupe:** [À COMPLÉTER]

---

## Description

Réalisation d'un modèle CLIP (Contrastive Language-Image Pre-training) pour associer des images et des textes.

**Dataset:** Flickr - 4 classes ("bike", "ball", "water", "dog") - 600 paires image-texte

## 📚 Importations

In [ ]:
# Imports standards
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models

# Transformers (pour SmallBERT)
from transformers import AutoTokenizer, AutoModel

# PIL pour les images
from PIL import Image

# Utilitaires projet
import sys
sys.path.append('../utils')
from config import *
from utils import *

# Configuration
set_seed(RANDOM_SEED)
print(f"Device: {DEVICE}")

## 📊 Chargement et Exploration des Données

In [ ]:
# TODO: Charger les données Flickr depuis ProjetClip.ipynb
# - Images: 4 classes x 150 images
# - Textes: captions associées

# Exemple de structure de données attendue:
# data = {
#     'image_path': [...],
#     'caption': [...],
#     'label': [...]  # 0: bike, 1: ball, 2: water, 3: dog
# }

---
# ✅ ÉTAPE 1: Classifieur CNN pour Images (4 classes)

## 1.1 Dataset et DataLoader

In [ ]:
class ImageDataset(Dataset):
    """Dataset pour les images Flickr"""
    
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # TODO: Implémenter le chargement d'image
        pass

# Transformations pour les images
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# TODO: Créer les DataLoaders

## 1.2 Architecture CNN

In [ ]:
class SimpleCNN(nn.Module):
    """CNN simple pour classification d'images"""
    
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        
        # TODO: Définir l'architecture
        # Suggestion: Conv2D -> ReLU -> MaxPool -> ... -> Flatten -> Dense
        pass
    
    def forward(self, x):
        # TODO: Implémenter le forward pass
        pass

# Créer le modèle
cnn_model = SimpleCNN(num_classes=NUM_CLASSES).to(DEVICE)
print(f"Nombre de paramètres: {count_parameters(cnn_model):,}")

## 1.3 Entraînement CNN

In [ ]:
# TODO: Entraîner le CNN
# ATTENTION: Ne pas perdre de temps à optimiser, juste un modèle fonctionnel

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=CNN_CONFIG['learning_rate'])

# Boucle d'entraînement
# ...

## 1.4 Évaluation CNN

In [ ]:
# TODO: Évaluer le CNN sur le test set
# Afficher: accuracy, confusion matrix, exemples de prédictions

---
# ✅ ÉTAPE 2: Classifieur SmallBERT pour Textes (4 classes)

## 2.1 Dataset et DataLoader

In [ ]:
# Charger SmallBERT
tokenizer = AutoTokenizer.from_pretrained(SMALLBERT_CONFIG['model_name'])
smallbert_base = AutoModel.from_pretrained(SMALLBERT_CONFIG['model_name'])

class TextDataset(Dataset):
    """Dataset pour les textes Flickr"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        # TODO: Tokenizer le texte
        pass

# TODO: Créer les DataLoaders

## 2.2 Architecture SmallBERT Classifier

In [ ]:
class SmallBERTClassifier(nn.Module):
    """Classifieur basé sur SmallBERT"""
    
    def __init__(self, smallbert_model, num_classes=4, hidden_size=512):
        super(SmallBERTClassifier, self).__init__()
        
        self.bert = smallbert_model
        
        # ATTENTION: SmallBERT n'a PAS de token <CLS>
        # TODO: Comment résumer la phrase?
        # Options:
        # 1. Mean pooling sur tous les tokens
        # 2. Max pooling
        # 3. Utiliser le dernier hidden state
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, input_ids, attention_mask):
        # TODO: Implémenter le forward pass
        pass

# Créer le modèle
smallbert_classifier = SmallBERTClassifier(smallbert_base, NUM_CLASSES).to(DEVICE)
print(f"Nombre de paramètres: {count_parameters(smallbert_classifier):,}")

## 2.3 Entraînement SmallBERT

In [ ]:
# TODO: Entraîner le modèle SmallBERT
# ATTENTION: Ne pas perdre de temps à optimiser

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(smallbert_classifier.parameters(), 
                        lr=SMALLBERT_CONFIG['learning_rate'])

# Boucle d'entraînement
# ...

## 2.4 Évaluation SmallBERT

In [ ]:
# TODO: Évaluer le classifieur de textes

---
# ✅ ÉTAPE 3: Modèle CLIP (CŒUR DU PROJET)

## 3.1 Encodeur Image (CNN sans classification)

In [ ]:
class ImageEncoder(nn.Module):
    """Encodeur image pour CLIP"""
    
    def __init__(self, cnn_model, embedding_dim=512):
        super(ImageEncoder, self).__init__()
        
        # TODO: Retirer les couches de classification du CNN
        # Garder jusqu'au flatten inclus
        
        # Projection vers l'espace d'embeddings
        # SANS fonction d'activation
        self.projection = nn.Linear(???, embedding_dim)
    
    def forward(self, x):
        # TODO: Implémenter le forward
        # N'OUBLIEZ PAS: normaliser la sortie
        pass

# Créer l'encodeur image
image_encoder = ImageEncoder(cnn_model, CLIP_CONFIG['embedding_dim']).to(DEVICE)
print(f"Image Encoder - Paramètres: {count_parameters(image_encoder):,}")

## 3.2 Encodeur Texte (SmallBERT sans classification)

In [ ]:
class TextEncoder(nn.Module):
    """Encodeur texte pour CLIP"""
    
    def __init__(self, bert_model, embedding_dim=512, hidden_size=512):
        super(TextEncoder, self).__init__()
        
        self.bert = bert_model
        
        # Projection vers l'espace d'embeddings
        # SANS fonction d'activation
        self.projection = nn.Linear(hidden_size, embedding_dim)
    
    def forward(self, input_ids, attention_mask):
        # TODO: Implémenter le forward
        # Résumer la phrase (mean pooling, etc.)
        # N'OUBLIEZ PAS: normaliser la sortie
        pass

# Créer l'encodeur texte
text_encoder = TextEncoder(smallbert_base, CLIP_CONFIG['embedding_dim']).to(DEVICE)
print(f"Text Encoder - Paramètres: {count_parameters(text_encoder):,}")

## 3.3 Modèle CLIP Complet

In [ ]:
class CLIPModel(nn.Module):
    """Modèle CLIP combinant image et texte"""
    
    def __init__(self, image_encoder, text_encoder):
        super(CLIPModel, self).__init__()
        
        self.image_encoder = image_encoder
        self.text_encoder = text_encoder
    
    def forward(self, images, input_ids, attention_mask):
        # Encoder les images et les textes
        image_embeddings = self.image_encoder(images)
        text_embeddings = self.text_encoder(input_ids, attention_mask)
        
        return image_embeddings, text_embeddings

# Créer le modèle CLIP
clip_model = CLIPModel(image_encoder, text_encoder).to(DEVICE)
print(f"CLIP Model - Total Paramètres: {count_parameters(clip_model):,}")

# Vérifier que les dimensions correspondent
print(f"\nDimension embeddings: {CLIP_CONFIG['embedding_dim']}")
print("✅ Les dimensions doivent être identiques pour image et texte!")

## 3.4 Loss Contrastive

In [ ]:
# La loss contrastive est déjà définie dans utils.py
contrastive_loss = ContrastiveLoss(temperature=CLIP_CONFIG['temperature'])

print("Loss Contrastive initialisée")
print(f"Température: {CLIP_CONFIG['temperature']}")

## 3.5 Dataset CLIP

In [ ]:
class CLIPDataset(Dataset):
    """Dataset pour CLIP (image + texte)"""
    
    def __init__(self, image_paths, captions, tokenizer, transform=None, max_length=128):
        self.image_paths = image_paths
        self.captions = captions
        self.tokenizer = tokenizer
        self.transform = transform
        self.max_length = max_length
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # TODO: Charger image + tokenizer texte
        pass

# TODO: Créer les DataLoaders CLIP

## 3.6 Entraînement CLIP

In [ ]:
# TODO: Entraîner le modèle CLIP

optimizer = optim.Adam(clip_model.parameters(), lr=CLIP_CONFIG['learning_rate'])

# Boucle d'entraînement
for epoch in range(CLIP_CONFIG['num_epochs']):
    # TODO: Training loop
    pass

# CRITIQUE: Vérifier la sauvegarde/rechargement
save_model(clip_model, PATHS['clip_model'])
clip_model, _ = load_model(clip_model, PATHS['clip_model'])
print("✅ Sauvegarde/rechargement testé avec succès!")

## 3.7 Inférence CLIP

### 3.7.1 Texte → Images (Top-5)

In [ ]:
def text_to_images(text_query, clip_model, image_dataset, tokenizer, top_k=5):
    """
    Trouve les top-k images correspondant au texte
    
    Returns:
        List[(image, score)]: Top-k images avec leurs scores
    """
    # TODO: Implémenter l'inférence texte → images
    # 1. Encoder le texte
    # 2. Encoder toutes les images
    # 3. Calculer similarités
    # 4. Retourner top-k avec scores
    pass

# Test
query = "A big dog in the woods"
results = text_to_images(query, clip_model, test_images, tokenizer)
display_top_k_results(query, results, query_type="text")

### 3.7.2 Image → Textes (Top-5)

In [ ]:
def image_to_texts(image, clip_model, text_dataset, top_k=5):
    """
    Trouve les top-k textes correspondant à l'image
    
    Returns:
        List[(text, score)]: Top-k textes avec leurs scores
    """
    # TODO: Implémenter l'inférence image → textes
    pass

# Test
test_image = # TODO: Charger une image de test
results = image_to_texts(test_image, clip_model, test_captions)
display_top_k_results(test_image, results, query_type="image")

---
# 🔧 TRAVAIL FACULTATIF

## Option A: Remplacer SmallBERT par DistilBERT

In [ ]:
# TODO (facultatif): Utiliser DistilBERT
# from transformers import DistilBertTokenizerFast, DistilBertModel
# ATTENTION: Vérifier l'alignement des dimensions

## Option B: Enrichir les textes courts via LLM

In [ ]:
# TODO (facultatif): Enrichir les captions courtes
# Conserver la même sémantique
# Les textes sont dans caption.csv et dans le répertoire caption/

---
# 📊 Résultats et Analyse

In [ ]:
# TODO: Présenter les résultats
# - Courbes d'entraînement
# - Exemples de requêtes texte → images
# - Exemples de requêtes image → textes
# - Analyse qualitative des résultats

---
# ✅ CHECKLIST FINALE

Avant de rendre le projet, vérifier:

- [ ] Nom, prénom, n° carte étudiant de tous les membres
- [ ] CNN fonctionnel (étape 1)
- [ ] SmallBERT fonctionnel (étape 2)
- [ ] Modèle CLIP complet (étape 3)
- [ ] Dimensions embeddings identiques (image et texte)
- [ ] Normalisation des embeddings activée
- [ ] Loss contrastive intégrée
- [ ] Sauvegarde/rechargement testé
- [ ] Inférence texte → images (top-5 + scores)
- [ ] Inférence image → textes (top-5 + scores)
- [ ] Fichiers nommés: `[GROUPE]_*.ipynb` et `[GROUPE]_*.pdf`
- [ ] Rapport LaTeX ≤ 8 pages (+ ≤ 2 pages annexes)
- [ ] Archive `[GROUPE].zip` avec tous les fichiers